# Capacitance matrix and LOM analysis
### Prerequisite
You need to have a working local installation of Ansys.

## 1. Create the design in Metal

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

In [3]:
design = designs.DesignPlanar()
gui = MetalGUI(design)

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander

In [4]:
design.variables['cpw_width'] = '15 um'
design.variables['cpw_gap'] = '9 um'

### In this example, the design consists of 4 qubits and 4 CPWs

In [24]:
# Allow running the same cell here multiple times to overwrite changes
design.overwrite_enabled = True

## Custom options for all the transmons
options = dict(
    # Some options we want to modify from the defaults
    # (see below for defaults)
    pad_width = '425 um', 
    pocket_height = '650um',
    # Adding 4 connectors (see below for defaults)
    connection_pads=dict(
        readout = dict(loc_W=+1,loc_H=-1, pad_width='200um'),
        bus1 = dict(loc_W=-1,loc_H=+1, pad_height='30um'),
        bus2 = dict(loc_W=-1,loc_H=-1, pad_height='50um')
    )
)
options2 = dict(
    # Some options we want to modify from the defaults
    # (see below for defaults)
    pad_width = '100 um', 
    pocket_height = '650um',
    pad_gap='100um', 
    # Adding 4 connectors (see below for defaults)
    connection_pads=dict(
        readout = dict(loc_W=+1,loc_H=-1, pad_width='10um'),
        #bus1 = dict(loc_W=-1,loc_H=+1, pad_height='10um'),
        #bus2 = dict(loc_W=-1,loc_H=-1, pad_height='10um')
    )
)
## Create 4 transmons

q1 = TransmonPocket(design, 'Q1', options = dict(
    pos_x='+2.42251mm', pos_y='+0.0mm',  **options2))
q2 = TransmonPocket(design, 'Q2', options = dict(
    pos_x='+0.0mm', pos_y='-0.95mm', orientation = '270', **options))
q3 = TransmonPocket(design, 'Q3', options = dict(
    pos_x='-2.42251mm', pos_y='+0.0mm', orientation = '180', **options))
q4 = TransmonPocket(design, 'Q4', options = dict(
    pos_x='+0.0mm', pos_y='+0.95mm', orientation = '90', **options))

RouteMeander.get_template_options(design)

options = Dict(
        lead=Dict(
            start_straight='0.2mm',
            end_straight='0.2mm'),
        trace_gap='9um',
        trace_width='15um')

def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, asymmetry='0 um', flip=False, fillet='90um'):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        fillet=fillet,
        hfss_wire_bonds = True,
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        total_length=length)
    myoptions.update(options)
    myoptions.meander.asymmetry = asymmetry
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

asym = 140
cpw1 = connect('cpw1', 'Q1', 'bus2', 'Q2', 'bus1', '6.0 mm', f'+{asym}um')
cpw2 = connect('cpw2', 'Q3', 'bus1', 'Q2', 'bus2', '6.1 mm', f'-{asym}um', flip=True)
cpw3 = connect('cpw3', 'Q3', 'bus2', 'Q4', 'bus1', '6.0 mm', f'+{asym}um')
cpw4 = connect('cpw4', 'Q1', 'bus1', 'Q4', 'bus2', '6.1 mm', f'-{asym}um', flip=True)

gui.rebuild()
gui.autoscale()

12:27AM 35s WARNING [__init__]: Pin bus2 does not exist in component Q1. cpw1 has not been built. Please check your pin_input values.
12:27AM 35s WARNING [__init__]: Pin bus1 does not exist in component Q1. cpw4 has not been built. Please check your pin_input values.


## 2. Capacitance Analysis and LOM derivation using the analysis package - most users

### Capacitance Analysis
Select the analysis you intend to run from the `qiskit_metal.analyses` collection.<br>
Select the design to analyze and the tool to use for any external simulation

In [6]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

(optional) You can review and update the Analysis default setup following the examples in the next two cells.

In [ ]:
c1.sim.setup

In [7]:
# example: update single setting
c1.sim.setup.max_passes = 6
# example: update multiple settings
c1.sim.setup_update(solution_order = 'Medium', auto_increase_solution_order = 'False')

c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': 5.0,
 'save_fields': False,
 'enabled': True,
 'max_passes': 6,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

Analyze a single qubit with 2 endcaps using the default (or edited) analysis setup. Then show the capacitance matrix (from the last pass).

You can use the method `run()` instead of `sim.run()` in the following cell if you want to run both cap extraction and lom analysis in a single step. If so, make sure to also tweak the setup for the lom analysis. The input parameters are otherwise the same for the two methods. 

In [26]:
c1.sim.run(components=['Q1'], open_terminations=[('Q1', 'readout')])
c1.sim.capacitance_matrix

INFO 12:29AM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
INFO 12:29AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 12:29AM [analyze]: Analyzing setup Setup
INFO 12:29AM [get_matrix]: Exporting matrix data to (C:\Users\WANGLAB\AppData\Local\Temp\tmpzq1c17s8.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 4800000000, Maxwell, 1, False
INFO 12:29AM [get_matrix]: Exporting matrix data to (C:\Users\WANGLAB\AppData\Local\Temp\tmpai7dbudz.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4800000000, Maxwell, 1, False
INFO 12:29AM [get_matrix]: Exporting matrix data to (C:\Users\WANGLAB\AppData\Local\Temp\tmpznoy2730.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4800000000, Maxwell, 2, False
INFO 12:29AM [get_matrix]: Exporting matrix data to (C:\Users\WANGLAB\AppData\Local\Temp\tmpq5kisv8k.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 

,ground_main_plane,pad_bot_Q1,pad_top_Q1,readout_connector_pad_Q1
ground_main_plane,140.41777,-18.00640,-20.30287,-39.82809
pad_bot_Q1,-18.00640,27.89593,-3.55537,-4.81338
pad_top_Q1,-20.30287,-3.55537,26.51273,-1.00425
readout_connector_pad_Q1,-39.82809,-4.81338,-1.00425,46.98324


(otional - case-dependent)<br>If the previous cell was interrupted due to license limitations and for any reason you finally manually launched the simulation from the renderer GUI (outside qiskit-metal) you might be able to recover the simulation results by uncommenting and executing the following cell

In [ ]:
#c1.sim._get_results_from_renderer()
#c1.sim.capacitance_matrix

The last variables you pass to the `run()` or `sim.run()` methods, will be stored in the `sim.setup` dictionary under the key `run`. You can recall the information passed by either accessing the dictionary directly, or by using the print handle below.

In [ ]:
# c1.setup.run    <- direct access
c1.sim.print_run_args()

You can re-run the analysis after varying the parameters.<br>
Not passing the parameter `components` to the `sim.run()` method, skips the rendering and tries to run the analysis on the latest design. If a design is not found, the full metal design is rendered.

In [9]:
c1.sim.setup.freq_ghz = 4.8
c1.sim.run()
c1.sim.capacitance_matrix

INFO 12:17AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 12:17AM [analyze]: Analyzing setup Setup
INFO 12:17AM [get_matrix]: Exporting matrix data to (C:\Users\WANGLAB\AppData\Local\Temp\tmp_p8olqr4.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 4800000000, Maxwell, 1, False
INFO 12:17AM [get_matrix]: Exporting matrix data to (C:\Users\WANGLAB\AppData\Local\Temp\tmp6q9yxxyd.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4800000000, Maxwell, 1, False
INFO 12:17AM [get_matrix]: Exporting matrix data to (C:\Users\WANGLAB\AppData\Local\Temp\tmpdd_p_hg2.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4800000000, Maxwell, 2, False
INFO 12:17AM [get_matrix]: Exporting matrix data to (C:\Users\WANGLAB\AppData\Local\Temp\tmpz8ngye3l.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 4800000000, Maxwell, 3, False
INFO 12:17AM [get_matrix]: Exporting matrix data to (C:\Users\WANGL

,bus1_connector_pad_Q1,bus2_connector_pad_Q1,ground_main_plane,pad_bot_Q1,pad_top_Q1,readout_connector_pad_Q1
bus1_connector_pad_Q1,49.77794,-0.42560,-33.50861,-1.57029,-13.15538,-0.20494
bus2_connector_pad_Q1,-0.42560,54.01885,-35.77522,-13.99741,-1.82852,-1.01319
ground_main_plane,-33.50861,-35.77522,237.69029,-31.53485,-37.88741,-36.55732
pad_bot_Q1,-1.57029,-13.99741,-31.53485,98.20667,-30.07382,-18.86490
pad_top_Q1,-13.15538,-1.82852,-37.88741,-30.07382,87.85076,-2.20122
readout_connector_pad_Q1,-0.20494,-1.01319,-36.55732,-18.86490,-2.20122,59.92347


In [ ]:
type(c1.sim.capacitance_matrix)

### Lumped oscillator model (LOM)

Using capacitance matrices obtained from each pass, save the many parameters of the Hamiltonian of the system. `get_lumped_oscillator()` operates on 4 setup parameters: <br><br>
Lj: float <br>
Cj: float <br>
fr: Union[list, float] <br>
fb: Union[list, float] <br>

In [32]:
c1.setup.junctions = Dict({'Lj': 30, 'Cj': 2})
c1.setup.freq_readout = 7.0
#c1.setup.freq_bus = [6.0]

c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.663519 [GHz]
EC 1168.702444 [MHz]
EJ 5.446520 [GHz]
alpha -992.735370 [MHz]
dispersion 1375124.014709 [KHz]
Lq 29.987903 [nH]
Cq 16.574132 [fF]
T1 114.579790 [us]

**Coupling Properties**

tCqbus1 1.804851 [fF]
gbus1_in_MHz 66.559194 [MHz]
χ_bus1 -2.885089 [MHz]
1/T1bus1 1389.031551 [Hz]
T1bus1 114.579790 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.798065,1245.655406,5.44652,-911.108286,1587717.998369,[64.00866745392301],[-2.988676056498704],2.988676,64.008667
2,5.806466,1250.54137,5.44652,-905.426571,1601490.721253,[63.16000167202934],[-2.9309539864423715],2.930954,63.160002
3,5.786886,1239.167802,5.44652,-918.564643,1569479.068003,[65.54061189868865],[-3.10367018882506],3.103670,65.540612
4,5.704067,1191.626715,5.44652,-970.024718,1437574.957023,[66.55783839361922],[-2.9843433831786403],2.984343,66.557838
5,5.693162,1185.438054,5.44652,-976.295305,1420639.280745,[66.24464886043923],[-2.929450973106334],2.929451,66.244649
6,5.663519,1168.702444,5.44652,-992.73537,1375124.014709,[66.55919412866066],[-2.8850890888614735],2.885089,66.559194


In [ ]:
c1.plot_convergence();
c1.plot_convergence_chi()

Once you are done with your analysis, please close it with `close()`. This will free up resources currently occupied by qiskit-metal to communiate with the tool.

In [ ]:
c1.sim.close()

## 3. Directly access the renderer to modify other parameters

In [ ]:
c1.sim.start()
c1.sim.renderer

Every renderer will have its own collection of methods. Below an example with q3d

##### Prepare and run a collection of predefined setups

This is equivalent to going to the Project Manager panel in Ansys, right clicking on Analysis within the active Q3D design, selecting "Add Solution Setup...", and choosing/entering default values in the resulting popup window. You might want to do this to keep track of different solution setups, giving each of them a different/specific name.

In [ ]:
setup = c1.sim.renderer.new_ansys_setup(name = "Setup_demo", max_passes = 6)

You can directly pass to `new_ansys_setup` all the setup parameters. Of course you will then need to run the individual setups by name as well.

In [ ]:
c1.sim.renderer.analyze_setup(setup.name)

##### Get the capactiance matrix at a different pass

You might want to use this if you intend to know what was the matrix at a different pass of the simulation.

In [ ]:
# Using the analysis results, get its capacitance matrix as a dataframe.
c1.sim.renderer.get_capacitance_matrix(variation = '', solution_kind = 'AdaptivePass', pass_number = 5)

### Code to swap rows and columns in capacitance matrix
from qiskit_metal.analyses.quantization.lumped_capacitive import df_reorder_matrix_basis

df_reorder_matrix_basis(fourq_q3d.get_capacitance_matrix(), 1, 2)

##### Close the renderer

In [ ]:
c1.sim.close()